#  Music taste prediction
Name - Akshat Sharma

Student Id - 220692379

Subject Code - MTH786P

Data Set - Spotify dataset

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 

### Load the data from the CSV file

In [2]:
df = pd.read_csv("Spotify_data.csv")
df.head()   # Display the first few rows of the data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,liked
0,0.803,0.6240,7,-6.764,0,0.0477,0.451,0.000734,0.1000,0.6280,95.968,304524,4,0
1,0.762,0.7030,10,-7.951,0,0.3060,0.206,0.000000,0.0912,0.5190,151.329,247178,4,1
2,0.261,0.0149,1,-27.528,1,0.0419,0.992,0.897000,0.1020,0.0382,75.296,286987,4,0
3,0.722,0.7360,3,-6.994,0,0.0585,0.431,0.000001,0.1230,0.5820,89.860,208920,4,1
4,0.787,0.5720,1,-7.516,1,0.2220,0.145,0.000000,0.0753,0.6470,155.117,179413,4,1


In [3]:
# Print the shape of the data
df.shape 

(195, 14)

In [4]:
X = df.drop(['liked'],axis='columns') # all columns except 'liked'
X.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.803,0.6240,7,-6.764,0,0.0477,0.451,0.000734,0.1000,0.6280,95.968,304524,4
1,0.762,0.7030,10,-7.951,0,0.3060,0.206,0.000000,0.0912,0.5190,151.329,247178,4
2,0.261,0.0149,1,-27.528,1,0.0419,0.992,0.897000,0.1020,0.0382,75.296,286987,4


In [5]:
y = df.liked # the 'liked' column
y.head()

0    0
1    1
2    0
3    1
4    1
Name: liked, dtype: int64

### Split the data into a training set and a test set 

In [6]:
m = X.shape[0]
split = int(0.8 * m)  # use 80% of the data for training
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

In [7]:
# Standardize the input features
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

### Define the linear regression model 

Linear regression -
$$
y_{pred} = \mathbf{X}\mathbf{w} + b
$$

Gradient of the loss function -
$$
dw = \frac{1}{n} \cdot \mathbf{X}^T \left(\mathbf{y_{pred}} - \mathbf{y}\right), \quad db = \frac{1}{n}  \sum_{i=1}^{m} (y_{pred_i} - Y_i)
$$


In [8]:
def linear_regression(X, Y, lr, n_iters):
    n, m = X.shape  # number of rows and columns in X

    b = 0   # initialize bias to 0
    w = np.zeros(m)   # initialize weights to 0

    # Iteratively update the weights and bias
    for i in range(n_iters):
        
        # Compute the linear prediction
        y_pred = np.dot(X, w) + b
        
        # Compute the gradient of the loss function
        dw = (1/n) * np.dot(X.T, (y_pred - Y))
        db = (1/n) * np.sum(y_pred - Y)
        
        # Update the weights and bias using gradient descent
        w = w - lr * dw
        b = b - lr * db
        
    return w, b     # Return the final values of the weights and bias

w_lr, b_lr = linear_regression(X_train, y_train, 0.001, 10000)  # Train the model using linear regression


#### Define a function to compute the accuracy of the model 

In [9]:
def accuracy(X, Y, w, b):
    y_pred = np.dot(X, w) + b  # Make predictions using the model
    ss_res = np.sum((Y - y_pred) ** 2) # residual sum of squares
    ss_tot = np.sum((Y - np.mean(Y)) ** 2) # total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

r_squared = accuracy(X_test, y_test, w_lr, b_lr)
print("Accuracy of model: ", round(r_squared*100,2), "%")

Accuracy of model:  58.63 %


### Define the logistic regression model 

sigmoid function.
\begin{equation*} 
\sigma(x): = \frac{1}{1+\mathrm{e}^{-x}} \, , 
\end{equation*}

In [10]:
# Define the sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

####  Gradient Descent

The cost function compute the cross-entropy loss, which is defined as

$$
cost = -\frac{1}{m} \sum_{i=1}^m \left( Y_i \log(y_{pred_i}) + (1-Y_i)\log(1-y_{pred_i}) \right)
$$


Computing the gradients of the loss with respect to the weights and bias
$$
dw = \frac{1}{m}  X^T  (y_{pred} - Y),\quad db = \frac{1}{m}  \sum_{i=1}^{m} (y_{pred_i} - Y_i)
$$

In [11]:
def gd_fit(X, Y, lr, n_iters):
    n, m = X.shape  # number of rows and columns in X

    b = 0   # initialize bias to 0
    w = np.zeros(m)   # initialize weights to 0
    
    cost_list = []    # list to store costs at each iteration
    
    # Iteratively update the weights and bias
    for i in range(n_iters):
        
        # Compute the linear prediction and the predicted probability
        linear_pred = np.dot(X, w) + b
        y_pred = sigmoid(linear_pred)
        
        # Compute the cross-entropy loss
        cost = -(1/m)*np.sum( Y*np.log(y_pred) + (1-Y)*np.log(1-y_pred))
    
        # Compute the gradients of the loss with respect to the weights and bias
        dw = (1/m) * np.dot(X.T, (y_pred - Y))
        db = (1/m) * np.sum(y_pred - Y)
    
        # Update the weights and bias using the gradient descent algorithm
        w = w - lr*dw
        b = b - lr*db
        
        cost_list.append(cost)   # Append the cost to the list of costs
        
        # Print the cost after every 10% of the iterations
        if i % (n_iters/10) == 0:
            print("cost after ", i, "iteration is : ", cost)
        
    return w, b, cost_list     # Return the final values of the weights, bias, and list of costs

w, b, cost_list = gd_fit(X_train, y_train, 0.001, 10000)  # Train the model

cost after  0 iteration is :  8.317766166719345
cost after  1000 iteration is :  3.148927021376165
cost after  2000 iteration is :  2.778215392734835
cost after  3000 iteration is :  2.612997384301308
cost after  4000 iteration is :  2.51345960191949
cost after  5000 iteration is :  2.4443468889313698
cost after  6000 iteration is :  2.3922914955966528
cost after  7000 iteration is :  2.3510117047665338
cost after  8000 iteration is :  2.31711857877443
cost after  9000 iteration is :  2.288595423968332


####  Ridge Regularization

$$
\nabla L = X^T(y_{pred}-y) + \alpha \cdot w
$$

In [23]:
def logistic_ridge_regression(X, Y, alpha, lr, n_iters):
    n, m = X.shape  # number of rows and columns in X

    b = 0   # initialize bias to 0
    w = np.zeros(m)   # initialize weights to 0

    # Iteratively update the weights and bias
    for i in range(n_iters):
        
        # Compute the linear prediction and the predicted probability
        linear_pred = np.dot(X, w) + b
        y_pred = sigmoid(linear_pred)
        
        # Compute the gradient of the loss function with regularization term
        grad = np.dot(X.T, (y_pred - Y)) + alpha * w
        
        # Update the weights and bias using gradient descent
        w = w - lr * grad
        b = b - lr * np.mean(y_pred - Y)
        
    return w, b     # Return the final values of the weights and bias

w_rr, b_rr = logistic_ridge_regression(X_train, y_train, 10, 0.001, 10000)  # Train the model using logistic ridge regression


####  Lasso Regularization

$$
\nabla L = X^T(y_{pred} - y) + \alpha \cdot \mathrm{sign}(w)
$$

In [21]:
def logistic_lasso_regression(X, Y, alpha, lr, n_iters):
    n, m = X.shape  # number of rows and columns in X

    b = 0   # initialize bias to 0
    w = np.zeros(m)   # initialize weights to 0

    # Iteratively update the weights and bias
    for i in range(n_iters):
        
        # Compute the linear prediction and the predicted probability
        linear_pred = np.dot(X, w) + b
        y_pred = sigmoid(linear_pred)
        
        # Compute the gradient of the loss function with regularization term
        grad = np.dot(X.T, (y_pred - Y)) + alpha * np.sign(w)
        
        # Update the weights and bias using gradient descent
        w = w - lr * grad
        b = b - lr * np.mean(y_pred - Y)
        
    return w, b     # Return the final values of the weights and bias

w_lr, b_lr = logistic_lasso_regression(X_train, y_train, 10, 0.001, 10000)  # Train the model using logistic lasso regression

#### Define a function to compute the accuracy of the model 

In [14]:
def accuracy(x, y, w, b):
    linear_pred = np.dot(x, w.T) + b
    y_pred = sigmoid(linear_pred)
    
    # Convert probabilities to class labels (0 or 1)
    y_pred = y_pred > 0.5
    y_pred = np.array(y_pred, dtype='int64')
    
    # Calculate the accuracy as the percentage of correct predictions
    acc = (1 - np.sum(np.absolute(y_pred - y)) / y.shape[0]) * 100

    return print("Accuracy of our model is: ", round(acc, 2), "%")

In [15]:
accuracy(X_test, y_test, w, b)

Accuracy of our model is:  97.44 %


In [24]:
accuracy(X_test, y_test, w_rr, b_rr)

Accuracy of our model is:  92.31 %


In [22]:
accuracy(X_test, y_test, w_lr, b_lr)

Accuracy of our model is:  84.62 %


### Define a function to predict the class labels (0 or 1) 

In [26]:
def predict(X, w, b):
    linear_pred = np.dot(X, w) + b
    y_pred = sigmoid(linear_pred)
    if y_pred <= 0.5:
        return 0
    else:
        return 1

In [27]:
input_data = [0.803, 0.6240, 7, -6.764, 0, 0.0477, 0.451, 0.000734, 0.1000, 0.6280, 95.968, 304524, 4]
prediction = predict(input_data, w_lr, b_lr)
print(prediction)

0


<ipython-input-10-cc8aaaecdfc2>:3: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
